DAYMET Downscaler
=================
**Author:** Xavier R Nogueira

**Overview:** Uses DAYMET [single pixel extractor API](https://daymet.ornl.gov/web_services#single), in conjunction with [US Hourly precipitation gage data](https://www.ncei.noaa.gov/maps/hourly/?layers=001) to train a model using a model that is a concrete implementation of the machine learning Python protocol defined in `machine_learning/protocols.py`.

Note: The goal here is to develop a performant, but data source light, prediction model such that data access for any region is rapid, and model explainability remains high.

## Potential features
* DAYMET data - raw daily value columns:
    * `tmax` - maximum temperature.
    * `tmin` - minimum temperature.
    * `srad` - shortwave radiation.
    * `vp` - vapor pressure.
    * `swe` - snow-water equivalent.
    * `prcp` - precipitation.
    * `dayl` - daylength.
    * **Note:** DAYMET data will be resampled (method pending) such that it can match the resolution of our 30m elevation data.
* Elevation data - 3DEP DEMs from [`Py3DEP`](https://docs.hyriver.io/readme/py3dep.html) are used to make the following features.
    * `raw_z` - raw elevation above SL.
    * `rel_z` - elevation 
* Kriging prediction - use just the ground data to predict spatially

## Environment prep

In [229]:
# import base python libaries
import requests
import datetime
import gc
from typing import Tuple

# import data manipulation libraries
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
from shapely.geometry import Polygon

# import hyrivers libraries to access data
import pydaymet
import py3dep

# Pull in data

## Define temporal/spatial AOI

In [105]:
# define a state to use data from
STATE = 'pennsylvania'

# define a date range to pull daily values from - i.e. (str) '2005-01-25'
start = '2016-01-01'
end = '2021-12-31'

# covnert to datetime
start_dt = datetime.datetime.strptime(date_range[0], '%Y-%m-%d')
end_dt = datetime.datetime.strptime(date_range[-1], '%Y-%m-%d')
date_range = (start_dt, end_dt)

In [106]:
def get_state_bbox(
    state_name: str,
        ) -> Tuple[float, float, float, float]:
    """
    Get the bounding box of a US state in EPSG4326 given it's name
    :param place: (str) a valid state name in english.
    :returns: (tuple) a tuple w/ coordinates as floats i.e.,
        [[11.777, 53.7253321, -70.2695876, 7.2274985]].
    """
    # create url to pull openstreetmap data
    url_prefix = 'http://nominatim.openstreetmap.org/search?state='

    url = '{0}{1}{2}'.format(url_prefix, state_name.lower(), '&format=json&polygon=0')
    response = requests.get(url).json()[0]

    lst = response['boundingbox']
    coors = [float(i) for i in lst]
    return {
        'xmin': coors[-2],
        'xmax': coors[-1],
        'ymin': coors[0],
        'ymax': coors[1],
        }

In [107]:
%%time
bbox_dict = get_state_bbox(STATE)
display(bbox_dict)

{'xmin': -80.5210833,
 'xmax': -74.689672,
 'ymin': 39.7197662,
 'ymax': 42.5146891}

CPU times: total: 15.6 ms
Wall time: 492 ms


### Build a dictionary to map state names to FIPs codes

In [108]:
# use the census api to get state names as a list of lists
list_of_pairs = list(
    requests.get(
        r'https://api.census.gov/data/2010/dec/sf1?get=NAME&for=state:*'
    ).json()
)

state_fips_dict = {}
for pair in list_of_pairs:
    if pair[0] != 'NAME': state_fips_dict[pair[0].lower()] = str(pair[-1])
display(state_fips_dict)

{'alabama': '01',
 'alaska': '02',
 'arizona': '04',
 'arkansas': '05',
 'california': '06',
 'louisiana': '22',
 'kentucky': '21',
 'colorado': '08',
 'connecticut': '09',
 'delaware': '10',
 'district of columbia': '11',
 'florida': '12',
 'georgia': '13',
 'hawaii': '15',
 'idaho': '16',
 'illinois': '17',
 'indiana': '18',
 'iowa': '19',
 'kansas': '20',
 'maine': '23',
 'maryland': '24',
 'massachusetts': '25',
 'michigan': '26',
 'minnesota': '27',
 'mississippi': '28',
 'missouri': '29',
 'montana': '30',
 'nebraska': '31',
 'nevada': '32',
 'new hampshire': '33',
 'new jersey': '34',
 'new mexico': '35',
 'new york': '36',
 'north carolina': '37',
 'north dakota': '38',
 'ohio': '39',
 'oklahoma': '40',
 'oregon': '41',
 'pennsylvania': '42',
 'rhode island': '44',
 'south carolina': '45',
 'south dakota': '46',
 'tennessee': '47',
 'texas': '48',
 'utah': '49',
 'vermont': '50',
 'virginia': '51',
 'washington': '53',
 'west virginia': '54',
 'wisconsin': '55',
 'wyoming': '56

## Gather daily precipitation gage data
**Note:** Move this to `/access_data/` if it is nicely done.

### Find valid stations

In [129]:
def validate_date(
    response_dict: dict,
    date_range: Tuple[datetime.datetime, datetime.datetime],
    str_format: str = '%Y-%m-%d',
        ) -> bool:
    # get station date range
    min_date = datetime.datetime.strptime(response_dict['mindate'], str_format)
    max_date = datetime.datetime.strptime(response_dict['maxdate'], str_format)

    if min_date <= date_range[0]:
        if max_date >= date_range[-1]:
            return True
    else:
        return False

In [252]:
%%time
base_stations_url = r'https://www.ncei.noaa.gov/cdo-web/api/v2/stations?limit=1000&datasetid=GHCND'
state_addition = f'&locationid=FIPS:{state_fips_dict[STATE.lower()]}'

print(f'Fetching precipitation stations for state={STATE}')
stations_list = list(requests.get(
    base_stations_url + state_addition,
    headers={'token': 'DSsSMpVbUnklkSjSzeAZFfaNsKsvommI'},
    ).json()['results'])
print(f'{len(stations_list)} precipitation stations total in {STATE}')
stations_list = [i for i in stations_list if validate_date(i, date_range)]
print(f'{len(stations_list)} precipitation stations with valid min/max date ranges')

# convert to dataframe
stations_df = pd.DataFrame.from_records(
    stations_list,
    index='id',
    )
stations_df.rename(columns={'elevation': 'elevation_m'}, inplace=True)
stations_df.index.name = 'station_id'
del stations_list
stations_df.head(n=5)

Fetching precipitation stations for state=pennsylvania
1000 precipitation stations total in pennsylvania
208 precipitation stations with valid min/max date ranges
CPU times: total: 15.6 ms
Wall time: 942 ms


,elevation_m,mindate,maxdate,latitude,name,datacoverage,elevationUnit,longitude
station_id,,,,,,,,
GHCND:US1PAAD0002,139.0,2007-05-20,2022-11-14,39.919185,"ABBOTTSTOWN 2.4 N, PA US",0.8369,METERS,-76.989416
GHCND:US1PAAD0003,189.6,1998-06-17,2022-11-12,40.001389,"YORK SPRINGS 0.7 SE, PA US",0.5041,METERS,-77.108889
GHCND:US1PAAD0006,360.6,2012-07-08,2022-11-13,39.889600,"ORRTANNA 3.2 NNW, PA US",0.8019,METERS,-77.375600
GHCND:US1PAAD0008,183.5,2012-08-19,2022-11-13,39.801061,"FAIRFIELD 1.0 NNE, PA US",0.8296,METERS,-77.360635
GHCND:US1PAAD0011,233.2,2012-08-25,2022-11-14,39.919780,"BIGLERVILLE 3.0 WSW, PA US",0.8144,METERS,-77.301310


### Access data
**NOTE:** Daily data is restricted to a one year range, and record limit is 1000, therefore API calls must be chunked.

In [224]:
years = list(range(start_dt.year, end_dt.year + 1))
years

[2016, 2017, 2018, 2019, 2020, 2021]

In [232]:
%%time
base_data_url = r'https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&limit=1000'

data_list = []
for year in years:
    print(f'Processing year={year}')
    for station_id in stations_df.index:
        station_id = str(station_id)
        url = base_data_url + state_addition + '&datatypeid=PRCP' + \
            f'&stationid={station_id}' + \
            f'&startdate={year}-01-01&enddate={year}-12-31'
        response = dict(requests.get(
            url,
            headers={'token': 'DSsSMpVbUnklkSjSzeAZFfaNsKsvommI'},
            ).json())
        if 'results' in list(response.keys()): data_list.extend(list(response['results']))
    gc.collect()

Processing year=2016
Processing year=2017
Processing year=2018
Processing year=2019
Processing year=2020
Processing year=2021
CPU times: total: 24.9 s
Wall time: 5min 31s


In [270]:
# construct a dataframe with all precipitation data
precip_df = pd.DataFrame().from_records(
    data_list,
    index='station',
    exclude=['attributes'],
)
precip_df.rename(columns={'value': 'precip_mm'}, inplace=True)
precip_df.drop(columns=['datatype'], inplace=True)
precip_df.index.name = 'station_id'

In [271]:
precip_df

,date,precip_mm
station_id,,
GHCND:US1PAAD0002,2016-02-28T00:00:00,0
GHCND:US1PAAD0002,2016-02-29T00:00:00,0
GHCND:US1PAAD0002,2016-03-01T00:00:00,0
GHCND:US1PAAD0002,2016-03-02T00:00:00,3
GHCND:US1PAAD0002,2016-03-03T00:00:00,0
...,...,...
GHCND:USC00363437,2021-11-23T00:00:00,0
GHCND:USC00363437,2021-11-24T00:00:00,0
GHCND:USC00363437,2021-11-25T00:00:00,0


### Join data to station information

In [272]:
station_info_df = stations_df[['elevation_m', 'latitude', 'longitude']]
station_info_df.head()

,elevation_m,latitude,longitude
station_id,,,
GHCND:US1PAAD0002,139.0,39.919185,-76.989416
GHCND:US1PAAD0003,189.6,40.001389,-77.108889
GHCND:US1PAAD0006,360.6,39.889600,-77.375600
GHCND:US1PAAD0008,183.5,39.801061,-77.360635
GHCND:US1PAAD0011,233.2,39.919780,-77.301310


In [277]:
try:
    precip_df = precip_df.join(station_info_df)
except ValueError:
    pass
precip_df['date'] = pd.to_datetime(precip_df['date'])
precip_df.info()
precip_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 296712 entries, GHCND:US1PAAD0002 to GHCND:USC00363437
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         296712 non-null  datetime64[ns]
 1   precip_mm    296712 non-null  int64         
 2   elevation_m  296712 non-null  float64       
 3   latitude     296712 non-null  float64       
 4   longitude    296712 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 21.6+ MB


,date,precip_mm,elevation_m,latitude,longitude
station_id,,,,,
GHCND:US1PAAD0002,2016-02-28,0,139.0,39.919185,-76.989416
GHCND:US1PAAD0002,2016-02-29,0,139.0,39.919185,-76.989416
GHCND:US1PAAD0002,2016-03-01,0,139.0,39.919185,-76.989416
GHCND:US1PAAD0002,2016-03-02,3,139.0,39.919185,-76.989416
GHCND:US1PAAD0002,2016-03-03,0,139.0,39.919185,-76.989416


In [280]:
from pathlib import Path
Path.cwd()

WindowsPath('C:/Users/xrnogueira/Documents/EnviroDataScience/machine_learning/analysis_notebooks')

## Pull in elevation data and build features
Here we do the following to calculate relative elevation `z_rel` compared to some radius `var:relative_z_radius`.
1. Resample our 30m 3DEP to 10, 25, and 50km using a mean and std algo.
2. Next us use nearest neighbors algo to resample both the mean and std back down to 30m.
2. With an elevation value, the regional cell's mean, and the cell's regional std we can calculate a z-score for each cell.
4. Finally we same this z-score raster at each precipitation gage point to create the `z_rel` column in our feature table.

## Pull in Daymet data and resample to match elevation data

# Build elevation featyres